## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:  

+ Assess situation:  

+ Determine data mining goals:  

+ Produce project plan:  



In [1]:
import pandas as pd
import basedosdados as bd

## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

### a. Collect initial data:  

In [2]:
# Define algumas variáveis úteis
dataset_id = "br_inep_indicadores_educacionais"
billing_project_id="trabalho-amd"

Agora visualizamos a estrutura do dataset

In [3]:
bd.list_dataset_tables(dataset_id=dataset_id)


table_id: 
	brasil 
-----------------------------------------------------------------------------------------------
table_id: 
	brasil_remuneracao_docentes 
-----------------------------------------------------------------------------------------------
table_id: 
	escola 
-----------------------------------------------------------------------------------------------
table_id: 
	escola_nivel_socioeconomico 
-----------------------------------------------------------------------------------------------
table_id: 
	fluxo_educacao_superior 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio 
-----------------------------------------------------------------------------------------------
table_id: 
	municipio_taxas_transicao 
-----------------------------------------------------------------------------------------------
table_id: 
	regiao 
--------------------------------------------------------------------------------------

Fazendo o download de cada uma das tabelas do dataset:

+ brasil

In [ ]:
bd.download(dataset_id=dataset_id, table_id='brasil', savepath='data/brasil.csv', billing_project_id=billing_project_id)

+ brasil_remuneracao_docentes

In [ ]:
bd.download(dataset_id=dataset_id, table_id='brasil_remuneracao_docentes', savepath='data/brasil_remuneracao_docentes.csv', billing_project_id=billing_project_id)

+ escola

In [ ]:
bd.download(dataset_id=dataset_id, table_id='escola', savepath='data/escola.csv', billing_project_id=billing_project_id)

+ escola_nivel_socioeconomico

In [22]:
bd.download(dataset_id=dataset_id, table_id='escola_nivel_socioeconomico', savepath='data/escola_nivel_socioeconomico.csv', billing_project_id=billing_project_id)


Downloading: 100%|██████████| 136830/136830 [00:18<00:00, 7490.88rows/s]


+ municipio

In [12]:
bd.download(dataset_id=dataset_id, table_id='municipio', savepath='data/municipio.csv', billing_project_id=billing_project_id)

Downloading: 100%|██████████| 1008636/1008636 [32:24<00:00, 518.80rows/s]


In [11]:
bd.download(dataset_id=dataset_id, table_id='municipio_taxas_transicao', savepath='data/municipio_taxas_transicao.csv', billing_project_id=billing_project_id)

Downloading: 100%|██████████| 262697/262697 [03:18<00:00, 1320.11rows/s]


In [ ]:
bd.download(dataset_id=dataset_id, table_id='regiao', savepath='data/regiao.csv', billing_project_id=billing_project_id)

In [9]:
bd.download(dataset_id=dataset_id, table_id='uf', savepath='data/uf.csv', billing_project_id=billing_project_id)

Downloading: 100%|██████████| 7141/7141 [00:15<00:00, 469.92rows/s]


In [10]:
bd.download(dataset_id=dataset_id, table_id='uf_remuneracao_docentes', savepath='data/uf_remuneracao_docentes.csv', billing_project_id=billing_project_id)

Downloading: 100%|██████████| 972/972 [00:00<00:00, 1879.55rows/s]


### b. Describe Data

In [ ]:
df.head(10)

In [ ]:
df.info()

### Explore Data

In [36]:
from matplotlib import pyplot as plt
import seaborn as sns

#### Peso vs. Altura

In [ ]:
colors = {'M':'blue', 'F':'red'}
plt.scatter(y = df['Height'], x = df['Weight'], alpha = 0.1, c=df['Sex'].map(colors))
# sns.jointplot(df['Height'], df['Weight'])

plt.ylabel('Altura dos Atletas')
plt.xlabel('Peso dos Atletas')
plt.rcParams.update({'figure.figsize':(10,10), 'figure.dpi':100})
plt.show()

In [ ]:
plt.clf()

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 